# Import Commands

In [1]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
import openpyxl
from qcodes.math_utils import FieldVector
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot

from qcodes_contrib_drivers.drivers.Tektronix.Keithley_2700 import Keithley_2700
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.instrument.visa import VisaInstrument
from qcodes_contrib_drivers.drivers.StanfordResearchSystems.SIM928 import SIM928
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.dataset.plotting import plot_dataset
from qcodes.instrument_drivers.oxford.MercuryiPS_VISA import MercuryiPS

# Initialize Instruments

In [2]:
#Magnet Controller
#mag120 = OxfordInstruments_IPS120('ips','TCPIP0::169.254.255.2::7020::SOCKET',23)
#mag120 = OxfordInstruments_IPS120('ips','GPIB0::23::INSTR')
mag120 = MercuryiPS('mips', 'TCPIP0::169.254.255.2::7020::SOCKET')
#Lockins(SR860 and SR830)
rxy1=SR860('Laughlin', 'GPIB1::4::INSTR')
rxx1=SR860('Kiwi','GPIB1::9::INSTR' )
rxy2=SR860('Pangolin', 'GPIB1::12::INSTR')
rxx2=SR830('Wally','GPIB1::1::INSTR' )
#SIM900 Controller(DC Voltage source)
#sim=SIM928('sim','GPIB1::15::INSTR')

Connected to: OXFORD INSTRUMENTS MERCURY IPS (serial:190250049, firmware:2.5.09.000) in 0.49s
Connected to: Stanford_Research_Systems SR860 (serial:004479, firmware:V1.51) in 0.15s
Connected to: Stanford_Research_Systems SR860 (serial:003172, firmware:V1.47) in 0.05s
Connected to: Stanford_Research_Systems SR860 (serial:004478, firmware:V1.51) in 0.05s
Connected to: Stanford_Research_Systems SR830 (serial:s/n48240, firmware:ver1.07) in 0.14s


C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch1_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,
C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch2_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,


# Set Parameters for Instruments

In [7]:
path = "C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11B4\\HallTransport\\BothHallBarsRxy&RxxSweepUp.xlsx"
amp = 1000000

# Set Up Database for Data Collection

In [ ]:
initialise_or_create_database_at('C:/Users/Sara/DataBases/M12-09-20.1SM1/M12-09-20.1SM1.db')

# Magnetic Field Code

In [8]:
from qcodes.math_utils import FieldVector
field = mag120._get_ramp_rate()
print(field)
rate = FieldVector(0.0,0.0,0.00083)
print(rate)
mag120._set_ramp_rate(rate)

FieldVector(x=0.0, y=0.0, z=0.0008299999999999999)
FieldVector(x=0.0, y=0.0, z=0.00083)


In [12]:
target = FieldVector(0.0,0.0,7.5)
print(mag120._get_target_field())
mag120._set_target_field(target)
print(mag120._get_target_field())

FieldVector(x=0.0, y=0.0, z=7.5)
FieldVector(x=0.0, y=0.0, z=7.5)


# Sweep Magnetic Field and Measure Results for 2 Hall Bars

In [14]:
workbook = openpyxl.Workbook()

sheet = workbook.active 

c1 = sheet.cell(row = 1, column = 1) 
c1.value = "Mag Field"
c2 = sheet.cell(row = 1, column = 2) 
c2.value = "Rxx(GPIB::4)"
c3 = sheet.cell(row = 1, column = 3) 
c3.value = "Rxx(GPIB::9)"
c4 = sheet.cell(row = 1, column = 4) 
c4.value = "Rxy(GPIB::12)"
c5 = sheet.cell(row = 1, column = 5) 
c5.value = "Rxx(GPIB::1)"
curr = 2

mag120.ramp('simul')
while(mag120._get_field().z<7.45):
    temp1 = sheet.cell(row = curr, column = 1)
    temp2 = sheet.cell(row = curr, column = 2)
    temp3 = sheet.cell(row = curr, column = 3)
    temp4 = sheet.cell(row = curr, column = 4)
    temp5 = sheet.cell(row = curr, column = 5)
    time.sleep(1)
    (X1, P1) = rxy1.get_values('X','P')
    print((X1, P1))
    (X2, P2) = rxx1.get_values('X','P')
    print((X2, P2))
    (X3, P3) = rxy2.get_values('X','P')
    print((X3, P3))
    (X4, P4)= rxx2.snap('x','p')
    print((X4, P4))
    print("\n")
    temp1.value = mag120._get_field().z
    temp2.value = X1*amp
    temp3.value = X2
    temp4.value = X3*amp
    temp5.value = X4
    curr = curr + 1
    workbook.save(filename=path)

(2.4178066269e-06, -8.7718849182)
(1.2876774235e-05, 1.1109421253)
(3.4570777643e-05, 1.2492790222)
(6.10356e-05, 0.0)


(3.6516400996e-06, -4.8899712563)
(1.2932790014e-05, 1.2243710756)
(3.5885528632e-05, 0.94619631767)
(6.10356e-05, 0.0)


(6.7245450737e-06, -3.3403584957)
(1.3005751498e-05, 1.3748743534)
(3.8984391722e-05, 1.1522682905)
(6.10356e-05, 0.0)


(1.0171615941e-05, -3.0710132122)
(1.3006420886e-05, 1.4430105686)
(4.264994277e-05, 1.229765296)
(6.10356e-05, 0.0)


(1.3550486074e-05, -2.7470371723)
(1.3001415937e-05, 1.1526763439)
(4.618073217e-05, 1.2570825815)
(6.10356e-05, 0.0)


(1.7014952391e-05, -2.5770030022)
(1.3054122974e-05, 0.99564003944)
(4.9625657994e-05, 1.164449811)
(6.10356e-05, 0.0)


(2.0295070499e-05, -2.1439056396)
(1.3085830687e-05, 0.76413083076)
(5.3136449424e-05, 1.1751646996)
(6.10356e-05, 0.0)


(2.390969712e-05, -2.2323260307)
(1.3100523574e-05, 0.70426416397)
(5.6796096032e-05, 1.3307052851)
(6.10356e-05, 0.0)


(2.7230053092e-05, -2.0499956608)

(0.00025441710022, -1.7214125395)
(1.3247392417e-05, -1.206607461)
(0.00028518631007, 1.5725997686)
(6.10356e-05, 0.0)


(0.00025796936825, -1.7372124195)
(1.3223348105e-05, -0.98805004358)
(0.00028870708775, 1.5847063065)
(6.10356e-05, 0.0)


(0.00026222490123, -1.725766778)
(1.3256452803e-05, -0.88336884975)
(0.00029278072179, 1.58515203)
(6.10356e-05, 0.0)


(0.00026582629653, -1.7322622538)
(1.3249304175e-05, -0.97561728954)
(0.00029615685344, 1.5824642181)
(6.10356e-05, 0.0)


(0.00026944206911, -1.744056344)
(1.3328765817e-05, -1.2780153751)
(0.00029958991217, 1.5721882582)
(6.10356e-05, 0.0)


(0.00027295065229, -1.7379962206)
(1.3152741303e-05, -1.2442896366)
(0.00030316747143, 1.6085555553)
(6.10356e-05, 0.0)


(0.00027708930429, -1.7461820841)
(1.3238417523e-05, -1.2382125854)
(0.00030694453744, 1.5984195471)
(6.10356e-05, 0.0)


(0.00028087155079, -1.7681651115)
(1.3306232177e-05, -1.1208561659)
(0.00031088406104, 1.600892067)
(6.10356e-05, 0.0)


(0.00028452670085, -1.74794

(0.00052148877876, -1.8269457817)
(1.0657401617e-05, -4.1434998512)
(0.00054860417731, 1.6023743153)
(6.10356e-05, 0.0)


(0.00052462273743, -1.8465312719)
(1.4914544408e-05, -3.0476098061)
(0.00055826612515, 1.616104722)
(6.10356e-05, 0.0)


(0.00052967731608, -1.8413814306)
(1.6344367396e-05, -2.7687315941)
(0.00055943161715, 1.6078964472)
(6.10356e-05, 0.0)


(0.00053478160407, -1.841821909)
(1.4230412489e-05, -3.1054992676)
(0.00055500637973, 1.615088582)
(0.0, 0.0)


(0.00053918082267, -1.8327889442)
(1.0144305634e-05, -3.83390975)
(0.00055271713063, 1.6132308245)
(0.0, 0.0)


(0.00054186122725, -1.8318021297)
(8.2520209617e-06, -4.5632081032)
(0.00056091544684, 1.6118016243)
(0.0, 0.0)


(0.0005441531539, -1.8548539877)
(1.1836626072e-05, -4.062526226)
(0.00057472841581, 1.6049817801)
(6.10356e-05, 0.0)


(0.00054691795958, -1.8603676558)
(1.5542955225e-05, -3.1712481976)
(0.00058190018171, 1.6233413219)
(6.10356e-05, 0.0)


(0.00055113219423, -1.8667402267)
(1.7174872482e-05, -2

(0.00078526337165, -1.989639163)
(2.6468615033e-05, -3.1145863533)
(0.00083503150381, 1.6098788977)
(6.10356e-05, 0.0)


(0.0007910230197, -1.9810382128)
(2.5389321308e-05, -3.1666891575)
(0.00083300052211, 1.6151227951)
(6.10356e-05, 0.0)


(0.00079780066153, -1.9731254578)
(2.2406329663e-05, -3.2600374222)
(0.00082775601186, 1.6099010706)
(6.10356e-05, 0.0)


(0.00080511689885, -1.9680591822)
(1.7660922822e-05, -4.2712225914)
(0.0008202008903, 1.6216593981)
(0.0, 0.0)


(0.00081076467177, -1.9463682175)
(1.1847158021e-05, -5.876894474)
(0.00081313401461, 1.6306649446)
(0.0, 0.0)


(0.00081472005695, -1.9261183739)
(5.9697013057e-06, -11.559531212)
(0.00081040262012, 1.6313290596)
(0.0, 0.0)


(0.00081647728803, -1.9342036247)
(2.7251437587e-06, -22.811794281)
(0.00081719364971, 1.624732852)
(0.0, 0.0)


(0.00081682449672, -1.9471862316)
(3.9650572035e-06, -17.709543228)
(0.00083084864309, 1.6186779737)
(0.0, 0.0)


(0.00081758323358, -1.9713975191)
(9.5385676104e-06, -8.0191745758)
(

(0.0010608934099, -2.1170802116)
(2.9965465728e-05, -3.6998960972)
(0.0011004550615, 1.6198954582)
(6.10356e-05, 0.0)


(0.0010679608677, -2.0997741222)
(2.5863782867e-05, -4.21773386)
(0.0010948072886, 1.6137943268)
(6.10356e-05, 0.0)


(0.0010749743087, -2.0851421356)
(2.0905117708e-05, -4.6181397438)
(0.0010879931506, 1.6254996061)
(0.0, 0.0)


(0.0010805390775, -2.0756397247)
(1.5839623302e-05, -6.2841086388)
(0.0010818786686, 1.6191234589)
(0.0, 0.0)


(0.001085277996, -2.0610334873)
(1.0017191016e-05, -9.7257413864)
(0.0010766995838, 1.6207184792)
(0.0, 0.0)


(0.0010883263312, -2.0592455864)
(5.0020917115e-06, -18.80134964)
(0.0010747051565, 1.6312711239)
(0.0, 0.0)


(0.0010899936315, -2.0496065617)
(1.4617065744e-06, -50.77879715)
(0.0010750072543, 1.6297923326)
(0.0, 0.0)


(0.0010904654628, -2.0470333099)
(3.3996579418e-07, -78.403831482)
(0.0010792500107, 1.6319864988)
(0.0, 0.0)


(0.0010901958449, -2.0573382378)
(6.9835158456e-07, -67.784011841)
(0.0010907072574, 1.624562

...

(0.023725157604, -14.372784615)
(0.00035399882472, -12.048911095)
(0.025190114975, 0.90884029865)
(0.000244142, -8.034)


(0.023742964491, -14.354426384)
(0.00034172675805, -12.225737572)
(0.025186443701, 0.91075444221)
(0.000244142, -7.83)


(0.023761987686, -14.334694862)
(0.00032858364284, -12.438358307)
(0.025181962177, 0.912142694)
(0.000183106, -8.454)


(0.023779604584, -14.31823349)
(0.00031684996793, -12.63934803)
(0.025178853422, 0.91305965185)
(0.000183106, -8.244)


(0.023798806593, -14.300580025)
(0.00030426878948, -12.875781059)
(0.025177208707, 0.91407048702)
(0.000183106, -8.88)


(0.023816403002, -14.284229279)
(0.000292549812, -13.109116554)
(0.025173539296, 0.9150352478)
(0.000183106, -9.618)


(0.023832064122, -14.268481255)
(0.00028080755146, -13.376569748)
(0.025171084329, 0.91633468866)
(0.000122071, -9.462)


(0.023848287761, -14.253156662)
(0.00026883688406, -13.684300423)
(0.025168795139, 0.9177775979)
(0.000122071, -10.206)


(0.023866049945, -14.238089561)
(

(0.02445882, -14.01500988)
(-2.0983348804e-05, -118.04165649)
(0.025142660365, 0.93446546793)
(0.0, -128.658)


(0.024462720379, -14.014854431)
(-2.1270956495e-05, -118.45883942)
(0.025142852217, 0.93396514654)
(0.0, -128.658)


(0.024465700611, -14.014890671)
(-2.1577108782e-05, -118.69740295)
(0.025142077357, 0.93457639217)
(0.0, -128.658)


(0.02446808666, -14.015090942)
(-2.1934654796e-05, -119.19499969)
(0.025142965838, 0.93360489607)
(0.0, -128.658)


(0.024470943958, -14.015306473)
(-2.219774251e-05, -119.43712616)
(0.025141933933, 0.93310964108)
(0.0, -128.658)


(0.024472525343, -14.01539135)
(-2.2324669771e-05, -119.62080383)
(0.025142595172, 0.93344092369)
(0.0, -123.69)


(0.02447427623, -14.015335083)
(-2.2513333533e-05, -119.83562469)
(0.025143017992, 0.93460720778)
(0.0, -128.658)


(0.024475153536, -14.01556015)
(-2.2611644454e-05, -119.95781708)
(0.025142390281, 0.93477112055)
(0.0, -128.658)


(0.024476263672, -14.015353203)
(-2.2802203603e-05, -120.12381744)
(0.02514

(0.024487527087, -14.017554283)
(-2.3358135877e-05, -120.71852112)
(0.025143055245, 0.93344604969)
(0.0, -128.658)


(0.024486972019, -14.017398834)
(-2.3542002964e-05, -120.89434814)
(0.02514315024, 0.93364584446)
(0.0, -128.658)


(0.024486621842, -14.017244339)
(-2.3403756131e-05, -120.74987793)
(0.025141773745, 0.93407952785)
(0.0, -128.658)


(0.024485982955, -14.01739502)
(-2.331077485e-05, -120.7024231)
(0.025143265724, 0.93345969915)
(0.0, -128.658)


(0.024485437199, -14.01748848)
(-2.3465750928e-05, -120.9439621)
(0.025142345577, 0.933157444)
(0.0, -128.658)


(0.024486122653, -14.017548561)
(-2.3348744435e-05, -120.69290161)
(0.025142569095, 0.93380463123)
(0.0, -123.69)


(0.024486435577, -14.017728806)
(-2.3454380425e-05, -120.85660553)
(0.025142690167, 0.934422791)
(0.0, -128.658)


(0.024484721944, -14.017550468)
(-2.3384543965e-05, -120.81282043)
(0.025143671781, 0.93447744846)
(0.0, -128.658)


(0.024486631155, -14.017842293)
(-2.3331229386e-05, -120.7364502)
(0.025142

(0.024488102645, -14.019224167)
(-2.3356462407e-05, -120.72834015)
(0.025143507868, 0.93388491869)
(0.0, -135.0)


(0.024486359209, -14.019713402)
(-2.3465325285e-05, -120.90253448)
(0.025143614039, 0.93331116438)
(0.0, -123.69)


(0.024486400187, -14.019306183)
(-2.3411472284e-05, -120.83551025)
(0.025142962113, 0.93352288008)
(0.0, -123.69)


(0.02448750101, -14.019310951)
(-2.3345250156e-05, -120.75823975)
(0.025143275037, 0.93370217085)
(0.0, -123.69)


(0.024486996233, -14.019486427)
(-2.3336420782e-05, -120.72207642)
(0.025142362341, 0.93294918537)
(0.0, -123.69)


(0.024487698451, -14.01941967)
(-2.3434215109e-05, -120.81549072)
(0.025141969323, 0.9327237606)
(0.0, -128.658)


(0.024487957358, -14.01941967)
(-2.3422042432e-05, -120.81651306)
(0.025141378865, 0.93306356668)
(0.0, -128.658)


(0.024486728013, -14.019234657)
(-2.3386062821e-05, -120.83731079)
(0.025143569335, 0.93388658762)
(0.0, -128.658)


(0.024487966672, -14.019504547)
(-2.3460472221e-05, -120.88365173)
(0.0251

(0.024485785514, -14.021469116)
(-2.3407428671e-05, -120.81842804)
(0.025142241269, 0.93253082037)
(0.0, -128.658)


(0.024485915899, -14.021792412)
(-2.3410577342e-05, -120.8031311)
(0.02514330484, 0.93293040991)
(0.0, -128.658)


(0.024484757334, -14.021634102)
(-2.3383518055e-05, -120.74253845)
(0.025142535567, 0.93362534046)
(0.0, -128.658)


(0.024485534057, -14.021650314)
(-2.3441920348e-05, -120.91352081)
(0.025143215433, 0.93308579922)
(0.0, -128.658)


(0.024486398324, -14.021377563)
(-2.3399377824e-05, -120.82198334)
(0.025143286213, 0.93371754885)
(0.0, -123.69)


(0.024488069117, -14.021537781)
(-2.3426166081e-05, -120.78287506)
(0.025143850595, 0.93459182978)
(0.0, -128.658)




In [4]:
station = qc.Station()
station.add_component(rxy1)
station.add_component(rxy2)
station.add_component(mag120)

'mips'

In [26]:
station.snapshot()

{'instruments': {'Zoe': {'functions': {'auto_range': {},
    'auto_scale': {},
    'auto_phase': {},
    'reset': {},
    'disable_front_panel': {},
    'enable_front_panel': {}},
   'submodules': {'data_channel_1': {'functions': {},
     'submodules': {},
     'parameters': {'assigned_parameter': {'__class__': 'qcodes.parameters.parameter.Parameter',
       'full_name': 'Zoe_data_channel_1_assigned_parameter',
       'value': 'X',
       'raw_value': '0',
       'ts': '2023-08-22 13:12:15',
       'inter_delay': 0,
       'unit': '',
       'instrument': 'qcodes.instrument_drivers.stanford_research.SR86x.SR86xDataChannel',
       'instrument_name': 'Zoe_data_channel_1',
       'post_delay': 0,
       'vals': "<Enum: {'aux_in4', 'aux_out1', 'amplitude', 'frequency_ext', 'sine_outdc', 'P', 'R', 'phase', 'Ynoise', 'aux_in3', 'aux_in2', 'Y', 'X', 'frequency', 'Xnoise', 'aux_out2', 'aux_in1'}>",
       'name': 'assigned_parameter',
       'val_mapping': {'X': '0',
        'Y': '1',
       

In [5]:
initialise_or_create_database_at("C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11C5\\Micro_Ohmic_Resistance\\Top&BottomMicroOhmicResistance2.db")

In [22]:
tutorial_exp = load_or_create_experiment(
    experiment_name="tutorial_exp",
    sample_name="synthetic data"
)
context_meas = Measurement(exp=tutorial_exp, station=station, name='context_example')
'''
#Independent Parameter
context_meas.register_parameter(mag120.field_measured)
#Dependent Parameter
context_meas.register_parameter(rxy1.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy1.P, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy2.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy2.P, setpoints=(mag120.field_measured,))
'''
context_meas.register_parameter(rxy1.X)
context_meas.register_parameter(rxy1.P)
context_meas.register_parameter(rxy2.X)
context_meas.register_parameter(rxy2.P)

In [23]:
context_meas.write_period = 2
target = FieldVector(0.0,0.0,0)
print(mag120._get_target_field())
mag120._set_target_field(target)

'''
(rxy1.X, rxy1.X()),
                         (rxy1.P,rxy1.P()),
                         (rxy2.X, rxy2.X()),
                         (rxy2.P,rxy2.P())
'''
mag120.ramp('simul')
with context_meas.run() as datasaver:
    while(mag120._get_field().z!=target.z):
        get_mag = mag120.field_measured.get()
        datasaver.add_result((rxy1.X, rxy1.X()),
                         (rxy1.P,rxy1.P()),
                         (rxy2.X, rxy2.X()),
                         (rxy2.P,rxy2.P()))

    # Convenient to have for plotting and data access
    dataset = datasaver.dataset

FieldVector(x=0.0, y=0.0, z=0.0)
Starting experimental run with id: 13. 


In [ ]:
dataset_1d = load_by_run_spec(experiment_name='tutorial_exp', captured_run_id=13)
df = dataset_1d.to_pandas_dataframe()